# Refs & Nexus Database


One of the core features of Quantinuum Nexus is a database for usefully structuring your scientific data.

`qnexus` aims to provide a useful way of interacting with this database through:

- `Ref` objects that can be used as a proxy object to 'reference' data like Circuits or Projects.

- `get` functions that take filters and try to return a single matching `Ref` 

- `get_all` functions that take filters and return `NexusIterator` objects for more than one matching`Ref`.  

In [ ]:
from datetime import datetime, timedelta

from pytket import Circuit

import qnexus as qnx

In [ ]:
# Get a 'ProjectRef' to a Project in Nexus, creating it if it doesn't exist
my_project_ref = qnx.projects.get_or_create(name="My Project")

In [ ]:
# Refs have a pandas DataFrame representation
my_project_ref.df()

In [ ]:
# Upload a circuit to Nexus, getting a CircuitRef in return
my_circuit_ref = qnx.circuits.upload(
    name=f"My Circuit from {datetime.now()}",
    circuit=Circuit(2).H(0).CX(0, 1).measure_all(),
    project=my_project_ref,
)

my_circuit_ref.df()

Refs point to underlying scientific data, in this case a pytket Circuit. We need to call a function to 'dereference' the CircuitRef and obtain the underlying circuit.

In [ ]:
my_circuit_ref.download_circuit()

We can perform queries to Nexus to retrieve all matching references.

In [ ]:
last_months_projects = qnx.projects.get_all(
    created_after=datetime.now() - timedelta(days=30)
)

This will return a NexusIterator, a Python object that represents our query. It behaves like a Python Iterator but with some additional functionality.

In [ ]:
# See how many projects match my query
last_months_projects.count()

In [ ]:
# Get the first ProjectRef
next(last_months_projects)

You can use other methods on the NexusIterator to get the ProjectRefs.

```python
# Get all the projects as a list
last_months_projects.list()

# Get all the projects as a DataFrame
last_months_projects.df()
```

If we think we can identify a single matching Ref to our query, we can use `get`. Bear in mind this will error if there is not exactly one matching Ref.

In [ ]:
qnx.projects.get(name_like="My Project").df()